In [2]:
import numpy as np
from numpy.linalg import inv
from numpy import matrix
from math import *
from sympy import *
x,y,z,w = symbols('x y z w')
init_printing(use_unicode=True)


## Integração numérica

Qualquer método de integração numérica *(ou quadratura numérica)* pode ser escrito como:

$$A = \sum_{j=1}^{N} f(x_i)w_i$$
 

-  x_i são as coordenadas dos pontos de integração
- w_i são os pesos, ou coeficientes

### Integração polinomial

Definimos os pesos da quadratura através da aproximação da função por um interpolador polinomial. Nesse caso, ainda, existem valores tabelados dos pontos/pesos que são resultados desse processo de interpolação e podemos usar.

In [144]:
def polynomialIntegration(N,interval,function,IntegrationTerm,show = False):
	a,b = interval[0],interval[1]
	if N == 1:
		x = [(a + b)/2]
	else:
		delta = (b - a)/(N - 1)
		x = []
		for i in range(N):
			x.append(a + i * delta)
	vandermonde = []
	for rowExpoent in range(N):
		row = []
		for column in range(N):
			row.append(x[column ]**rowExpoent)
		vandermonde.append(row)
	y = []
	for j in range(1,N+1):
		y.append([(b ** j - a ** j)/j])
	w = list((inv(vandermonde) * matrix(y)).flat)
	result = 0
	for i in range(N):
		result += w[i] * function.evalf(subs={IntegrationTerm:x[i]})
	return result

## Quadratura de Gauss

É uma aproximação da integral de uma função, geralmente estabelecido como um somatório com pesos dos valores assumidos pela função em pontos específicos dentro do domínio de integração. Possui um domínio de integração fixo [0,1].

In [4]:
def quadrature(numberOfpoints,interval,function,IntegrationTerm,show = False):
	"""
	Implements: Gauss` Quadrature
	Arguments:
		numberOfpoints: number of interpolation points (number)
		interval: range of integration (list)
		function: function to apply the methos (sympy function)
		IntegrationTerm: term to integrate the function (string)
		show : active DEBUG mode (boolean)
	Return: integral of analyzed function (number) 
	"""

	# Points is a list of lists
	# Each list contain the points to use when the user set N points
	points = []
	# Points [2,10] extracted from https://pomax.github.io/bezierinfo/legendre-gauss.html
	points.append([0])
	points.append([-0.5773502691896257,0.5773502691896257])
	points.append([0.0000000000000000,-0.7745966692414834,0.7745966692414834])
	points.append([-0.3399810435848563,0.3399810435848563,-0.8611363115940526,0.8611363115940526])
	points.append([0.0000000000000000,-0.5384693101056831,0.5384693101056831,-0.9061798459386640,0.9061798459386640])
	points.append([0.6612093864662645,-0.6612093864662645,-0.2386191860831969,0.2386191860831969,-0.9324695142031521,0.9324695142031521])
	points.append([0,0.4058451513773972,-0.4058451513773972,-0.7415311855993945,0.7415311855993945,-0.9491079123427585,0.9491079123427585])
	points.append([-0.1834346424956498,0.1834346424956498,-0.5255324099163290,0.5255324099163290,-0.7966664774136267,0.7966664774136267,-0.9602898564975363,0.9602898564975363])
	points.append([-0.8360311073266358,0.8360311073266358,-0.9681602395076261,0.9681602395076261,-0.3242534234038089,0.3242534234038089,-0.6133714327005904,0.6133714327005904])
	points.append([-0.1488743389816312,0.1488743389816312,-0.4333953941292472,0.4333953941292472,-0.6794095682990244,0.6794095682990244,-0.8650633666889845,0.8650633666889845,-0.9739065285171717,0.9739065285171717])
	
	elementIndex = numberOfpoints-1
	if(show):
		print("PONTOS: ",points[elementIndex])

	# Weights is a list of lists
	# Each list contain the weight to use when the user set N points
	# Weights extracted from https://pomax.github.io/bezierinfo/legendre-gauss.html

	weights = [[1],
			  [1.0000000000000000,1.0000000000000000],
			  [0.8888888888888888,0.5555555555555556,0.5555555555555556],
			  [0.6521451548625461,0.6521451548625461,0.3478548451374538,0.3478548451374538],
			  [0.5688888888888889,0.4786286704993665,0.4786286704993665,0.2369268850561891,0.2369268850561891],
			  [0.3607615730481386,0.3607615730481386,0.4679139345726910,0.4679139345726910,0.1713244923791704,0.1713244923791704],
			  [0.4179591836734694,0.3818300505051189,0.3818300505051189,0.3818300505051189,0.2797053914892766,0.1294849661688697,0.1294849661688697],
			  [0.3626837833783620,0.3626837833783620,0.3137066458778873,0.3137066458778873,0.2223810344533745,0.2223810344533745,0.1012285362903763,0.1012285362903763],
			  [0.3302393550012598,0.1806481606948574,0.1806481606948574,0.0812743883615744,0.0812743883615744,0.3123470770400029,0.3123470770400029,0.2606106964029354,0.2606106964029354],
			  [0.2955242247147529,0.2955242247147529,0.2692667193099963,0.2692667193099963,0.2190863625159820,0.2190863625159820,0.1494513491505806,0.1494513491505806,0.0666713443086881,0.0666713443086881]]
	
	selectWeightArray = weights[len(points[elementIndex])-1]
	result = 0;

	for i in range(len(points[elementIndex])):
		adjustTerm = (interval[-1]-interval[0])/2
		# Adjusts the function to work using the weights and points of Legendre-Gauss interval [-1,1]
		functionValue = function.subs(IntegrationTerm,adjustTerm*points[elementIndex][i] + (interval[-1]+interval[0])/2)
		weight = selectWeightArray[i]
		result += adjustTerm*(functionValue * weight)
		if(show):	
			print ("RESULTADO PARA ",i+1," PONTOS: ",result)
	return result


# Questão número 2

In [141]:
testFunction1 = exp((1/2)*-x**2)*((1/sqrt(2*np.pi)))
testFunction2 = exp((1/2)*-x**2)*((1/sqrt(2*np.pi)))
N = 10
print("RESULTADO EXATO DA FUNÇÃO 1: ", 0.341345)
print("RESULTADO EXATO DA FUNÇÃO 2: ", 0.5,"\n")

print("FUNÇÃO 1 (5 PONTOS)(Integração polinomial): ",polynomialIntegration(5,[0,1],testFunction1,x))
print("FUNÇÃO 1 (5 PONTOS)(Quadratura): ",quadrature(5,[0,1],testFunction1,x),"\n")

print("FUNÇÃO 2 (5 PONTOS)(Integração polinomial): ",polynomialIntegration(5,[0,5],testFunction2,x))
print("FUNÇÃO 2 (5 PONTOS)(Quadratura): ",quadrature(5,[0,5],testFunction2,x))



RESULTADO EXATO DA FUNÇÃO 1:  0.341345
RESULTADO EXATO DA FUNÇÃO 2:  0.5 

FUNÇÃO 1 (5 PONTOS)(Integração polinomial):  0.341343916914007
FUNÇÃO 1 (5 PONTOS)(Quadratura):  0.341344746063202 

FUNÇÃO 2 (5 PONTOS)(Integração polinomial):  0.492166653892020
FUNÇÃO 2 (5 PONTOS)(Quadratura):  0.500447447519548


Definimos a função **minPoints** com o bojetivo de achar o mínimo de pontos de integração possível para encontrar o resultado exato. *(consideramos que chegamos ao resultado exato quando, ao variar o número de ponto, mudamos nosso valor em menos de 0.00001)*

In [103]:
def minPoints(function,interval,variable,method):
    if(method == 0):
        result = polynomialIntegration(2,interval,function,variable)
        for i in range(3,11):
            i_result = polynomialIntegration(i,interval,function,variable)
            if (abs(i_result - result) < 0.00001):
                return i-1
            else:
                result = i_result
    if(method == 1):
        result = quadrature(2,interval,function,variable)
        for i in range(3,11):
            i_result = quadrature(i,interval,function,variable)
            if (abs(i_result - result) < 0.00001):
                return i-1
            else:
                result = i_result
    return 10

In [104]:
print(minPoints(testFunction1,[0,1],x,0))
print(minPoints(testFunction2,[0,1],x,1))

5
3


# Questão 3

In [171]:

RAO_2 = ((1/(sqrt((1-(w/1.0)**2)**2  +  (2*0.05*(w/1.0) )**2)))**2)
print(RAO_2.subs(w,1))
print(RAO_2.subs(w,0.9))
testFunction3 = RAO_2*2
# testFunction3 = exp(-(w**2))

# m0aux1 = integracao_polinomial(f,0, 1, 10)
# m0aux2 = integracao_polinomial(f,1, 2, 10)
# m0aux3 = integracao_polinomial(f,2, 10, 10)



testFunction4 = (w**2)*RAO_2*2

m0 = polynomialIntegration(10,[0,1],testFunction3,w) + polynomialIntegration(10,[1,2],testFunction3,w) +  polynomialIntegration(10,[2,10],testFunction3,w)
m0_q = quadrature(10,[0,1],testFunction3,w) + quadrature(10,[1,2],testFunction3,w) +  quadrature(10,[2,10],testFunction3,w)

m2 = polynomialIntegration(10,[0,1],testFunction4,w) + polynomialIntegration(10,[1,2],testFunction4,w) +  polynomialIntegration(10,[2,10],testFunction4,w)
m2_q = quadrature(10,[0,1],testFunction4,w) + quadrature(10,[1,2],testFunction4,w) +  quadrature(10,[2,10],testFunction4,w)

print("RESULTADO EXATO DA FUNÇÃO m0: ", 32.4153)
print("RESULTADO EXATO DA FUNÇÃO m2: ", 31.2146,"\n")



print("RESUTADO QUESTAO 3 - m0_polinomial: ",m0)
print("RESUTADO QUESTAO 3 - m0_quadratura: ",m0_q,"\n")

print("RESUTADO QUESTAO 3 - m2_polinomial: ",m2)
print("RESUTADO QUESTAO 3 - m2_quadratura: ",m2_q,"\n")

100.000000000000
22.6244343891403
RESULTADO EXATO DA FUNÇÃO m0:  32.4153
RESULTADO EXATO DA FUNÇÃO m2:  31.2146 

RESUTADO QUESTAO 3 - m0_polinomial:  29.0062788916839
RESUTADO QUESTAO 3 - m0_quadratura:  31.3744327374628 

RESUTADO QUESTAO 3 - m2_polinomial:  28.8104930627014
RESUTADO QUESTAO 3 - m2_quadratura:  31.1746007967469 



# Questão 4

In [163]:
Sn = ((4*(np.pi**3)*(3**2))/(w**5)*(5**4))* exp((-16*(np.pi**3))/((w**4)*(5**4)))

testFunction3 = RAO_2 *Sn
testFunction4 = (w**2)*RAO_2*Sn
m0_2 = polynomialIntegration(10,[0.00001,1],testFunction3,w) + polynomialIntegration(10,[1,2],testFunction3,w) +  polynomialIntegration(10,[2,10],testFunction3,w)
m0_2_q = quadrature(10,[0,1],testFunction3,w) + quadrature(10,[1,2],testFunction3,w) +  quadrature(10,[2,10],testFunction3,w)

m2_2 = polynomialIntegration(10,[0.00001,1],testFunction4,w) + polynomialIntegration(10,[1,2],testFunction4,w) +  polynomialIntegration(10,[2,10],testFunction4,w)
m2_2_q = quadrature(10,[0,1],testFunction4,w) + quadrature(10,[1,2],testFunction4,w) +  quadrature(10,[2,10],testFunction4,w)

#https://www.wolframalpha.com/input/?i=integrate(((1%2F(sqrt((1-(w%2F1.0)**2)**2++%2B++(2*0.05*(w%2F1.0)+)**2)))**2)*2)+++0+to+10


print("RESULTADO QUESTAO 4 m0_polinomia: ",m0_2)
print("RESULTADO QUESTAO 4 m0_quadrature: ",m0_2_q)
print("RESULTADO QUESTAO 4 m2_polinomial: ",m2_2)
print("RESULTADO QUESTAO 4 m2_quadrature: ",m2_2_q,"\n")


RESULTADO QUESTAO 4 m0_polinomia:  3994198.17316899
RESULTADO QUESTAO 4 m0_quadrature:  4379478.94871019
RESULTADO QUESTAO 4 m2_polinomial:  3906500.86405490
RESULTADO QUESTAO 4 m2_quadrature:  4289175.19750818 



# Questão 5

In [158]:
testFunction5 = 2 + 2*x - x**2 + 3*x**3
print("Integração polinomial resultou ",polynomialIntegration(minPoints(testFunction5,[0,4],x,0),[0,4],testFunction5,x) ," com ",minPoints(testFunction5,[0,1],x,0)," pontos de integração")
print("Integração por quadratura resultou ",quadrature(minPoints(testFunction5,[0,4],x,0),[0,4],testFunction5,x) ," com ",minPoints(testFunction5,[0,1],x,1)," pontos de integração")

Integração polinomial resultou  194.666666666667  com  3  pontos de integração
Integração por quadratura resultou  194.666666666667  com  2  pontos de integração


# Questão 6

In [153]:
testFunction6 = 1/(1+x**2)
interval = [0,3]

# Result using mid point rule
m = (interval[1]+interval[0])/2
A_mid = testFunction6.subs(x,m) * (interval[1]-interval[0])
print("RESULTADO DA TECNICA DO PONTO MÉDIO: ", A_mid)

# Result using trapeze rule
A_t = ((testFunction6.subs(x,interval[0])+testFunction6.subs(x,interval[1]))/2.0) * (interval[1]-interval[0])
print("RESULTADO DA TECNICA DO TRAPEZIO: ", A_t)

#Result using Simpson rule
S = (2/3)*A_mid + (1/3)*(A_t)
print("RESULTADO DA RESOLUCAO POR SIMPSON: ", S)
# Result using numeric method
A = polynomialIntegration(10,interval,testFunction6,x)
print("RESULTADO DA RESOLUCAO POR INTEGRACÃO POLINOMIAL: ", A)
B =  quadrature(10,interval,testFunction6,x)
print("RESULTADO DA RESOLUCAO POR INTEGRACÃO POLINOMIAL: ", B)

RESULTADO DA TECNICA DO PONTO MÉDIO:  0.923076923076923
RESULTADO DA TECNICA DO TRAPEZIO:  1.65000000000000
RESULTADO DA RESOLUCAO POR SIMPSON:  1.16538461538462
RESULTADO DA RESOLUCAO POR INTEGRACÃO POLINOMIAL:  1.24941630711226
RESULTADO DA RESOLUCAO POR INTEGRACÃO POLINOMIAL:  1.24904580825023
